In [1]:
%logstop
%logstart -rtq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [3]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [4]:
import numpy as np
import pandas as pd

In [5]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation
5,ZIP,Provider Zip Code,Provider Zip Code,5-digit zip code
6,PHONE,Provider Phone Number,Provider Phone Number,10 digit phone number
7,COUNTY_SSA,Provider SSA County,SSA county code,3-digit SSA code
8,COUNTY_NAME,Provider County Name,Provider County Name,text
9,OWNERSHIP,Ownership Type,Nature of organization that operates a provide...,text


In [6]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [7]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.avg =  0

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.avg = y.mean()
        return self

    def predict(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages.get(row, self.avg) 
                for row in X[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [9]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [10]:
state_model.predict(data.sample(5))

[18147.119744058502,
 29459.975,
 18425.87278106509,
 5626.954545454545,
 29459.975]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [11]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [12]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [13]:
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
])

Let's check the missing values of the feature columns

In [14]:
data[simple_cols].isnull().sum()

BEDCERT               0
RESTOT              409
INHOSP                0
CCRC_FACIL            0
SFF                   0
CHOW_LAST_12MOS       0
SPRINKLER_STATUS      0
EXP_TOTAL           788
ADJ_TOTAL           788
dtype: int64

From the above result, we can see that the columns RESTOT, EXP_TOTAL, ADJ_TOTAL have missing large number of observations(null values) and many of the columns are boolean data type. So, we will impute missing values using the `Imputer` method from the `sklearn`.

In [15]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='mean')
imp.fit_transform(data[simple_cols])

simple_features = Pipeline([('cst', ColumnSelectTransformer(simple_cols)),
                            ('imp', imp) ])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [16]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
grid_param = {"C": np.logspace(-2, 2, 10)}
grid_model = RandomizedSearchCV(LogisticRegression(solver='lbfgs'), grid_param, cv=5, verbose=2)

simple_features_model = Pipeline(steps=[('simple', simple_features), 
                                        ('model', grid_model)])

In [18]:
simple_features_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.2s
[CV] C=0.01 ..........................................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.027825594022071243 ..........................................


...

[CV] .......................................... C=100.0, total=   0.1s
[CV] C=100.0 .........................................................
[CV] .......................................... C=100.0, total=   0.1s
[CV] C=100.0 .........................................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV] .......................................... C=100.0, total=   0.2s
[CV] C=100.0 .........................................................
[CV] .......................................... C=100.0, total=   0.1s
[CV] C=100.0 .........................................................
[CV] .......................................... C=100.0, total=   0.1s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    6.8s finished
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imp',
                                  SimpleImputer(add_indicator=False, copy=True,
                           

In [19]:
simple_features_model.score(data, fine_counts>0)

0.6937086092715232

In [20]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.003954985078919


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [21]:
data[['OWNERSHIP', "CERTIFICATION"]].nunique()

OWNERSHIP        13
CERTIFICATION     3
dtype: int64

In [22]:
data[['OWNERSHIP', "CERTIFICATION"]].isnull().sum()

OWNERSHIP        0
CERTIFICATION    0
dtype: int64

In [23]:
from sklearn.pipeline import FeatureUnion
from  sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(categories='auto', sparse=False)
imp = SimpleImputer(strategy='most_frequent')
owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])), 
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

categorical_features = FeatureUnion([('owner_onehot', owner_onehot), ('cert_onehot', cert_onehot)
])

In [24]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [100, 250, 50]
max_depth = [2, 4, 6]
grid_param = {'max_depth': np.logspace(-2, 2, 10)}

grid_search_model = RandomizedSearchCV(RandomForestClassifier(n_estimators=200), grid_param, cv=5, n_jobs=2, n_iter=20, verbose=2)
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    
    ('classifier', grid_search_model)
])

In [26]:
categorical_features_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 10 is smaller than n_iter=20. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   18.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   27.0s finished


Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner_onehot',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories='auto',
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='er

In [27]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  0.9317667381462962


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [28]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [29]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
grid_param = {"C": np.logspace(-2, 2, 10)}
grid_model = RandomizedSearchCV(LogisticRegression(), grid_param, cv=5, verbose=2)

business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('model', grid_model)
])

In [30]:
business_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[CV] ........................................... C=0.01, total=   0.7s
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[CV] ........................................... C=0.01, total=   0.9s
[CV] C=0.01 ..........................................................


...

[CV] .......................................... C=100.0, total=   0.7s
[CV] C=100.0 .........................................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[CV] .......................................... C=100.0, total=   0.5s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   35.1s finished
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [31]:
business_model.score(data, fine_counts>0)

0.6976677224301756

In [32]:
grader.score.ml__business_model(positive_probability(business_model))

Your score:  1.0213720587731625


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [33]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance (X, pd.DataFrame):
            X = pd.DataFrame(X)
        results = (pd.to_datetime(X[self.t1_col]) - pd.to_datetime(X[self.t1_col]))
        
        results = results.apply(lambda X:X.days).values.reshape(-1, 1)
        
        return results

In [34]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [35]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Lasso

grid_search = GridSearchCV(Lasso(max_iter = 100), param_grid = {'alpha': np.arange(0, 3.5, 0.5)}, cv=5, n_jobs=4, verbose=1)
survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('decomp', TruncatedSVD(40)),
    ('grid_search', grid_search)
])

In [37]:
survey_model.fit(data, cycle_2_score.astype(int))

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:    5.0s finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [38]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  0.9801414702727278


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*